# Intro

In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import show
from jupyter_cadquery import set_defaults, set_sidecar #, close_sidecar, reset_sidecar, get_defaults, get_default

Overwriting auto display for cadquery Workplane and Shape


In [2]:
set_defaults(axes=True, axes0=True, grid=True)

In [ ]:
cq.Workplane('XY').box(10, 20, 30).edges(">X or <X").fillet(2)

# Assemblies

In [7]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar
set_defaults(width=650, height=650)
set_sidecar("CadQuery", init=True)

Paramater width is not a valid argument for show()


In [ ]:
enable_replay(warning=False)  # False disables the repeated warning about replay being enabled!

In [ ]:
box1 = cq.Workplane('XY').box(10, 20, 30).edges(">X or <X").chamfer(2)
box1.name = "box1"

box2 = cq.Workplane('XY').box(8, 18, 28).edges(">X or <X").chamfer(2)
box2.name = "box2"

box3 = cq.Workplane('XY').transformed(offset=(0, 15, 7))\
    .box(30, 20, 6).edges(">Z").fillet(3)
box3.name = "box3"

box4 = box3.mirror("XY").translate((0, -5, 0))
box4.name = "box4"

box1 = box1\
    .cut(box2)\
    .cut(box3)\
    .cut(box4)

In [ ]:
%%time
a1 = PartGroup(
    [
        Part(box1, "red box",   "#d7191c", show_edges=False),
        Part(box3, "green box", "#abdda4", show_edges=False),
        Part(box4, "blue box",  "#2b83ba", show_faces=False),
    ],
    "example 1"
)
d = show(a1, axes=True, grid=True, ortho=False, axes0=True)

# Faces, Edges, Vertices

In [ ]:
box1

In [ ]:
box1.faces("not(|Z or |X or |Y)")

In [ ]:
box1.edges("not(|X or |Y or |Z)")

In [ ]:
box1.vertices()

In [ ]:
# Replay

In [ ]:
r = replay(box1)

# Export to HTML

In [ ]:
from ipywidgets.embed import embed_minimal_html, dependency_state
embed_minimal_html(
    'export.html', 
    title='Box', 
    views=[d.cq_view.renderer], 
    state=dependency_state(d.cq_view.renderer))